<a href="https://colab.research.google.com/github/kenrp8/Coursera_Capstone/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML 


In [4]:
# Read the CVS coordenates data from given URL
post_code_coord = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_data.cvs')
post_code_coord.head()
#print('data shape:', post_code_coord.shape)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now we use the first part of this assignment to get __Borough__ and __Neighborhood__ from each code.

In [5]:
# Now we are interested in get the Postal Code info from Wikipedia

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html_doc=requests.get(url).text

#print(html_doc) #this print the raw html

soup = BeautifulSoup(html_doc, 'html.parser')

#To get the table, We can use the given hint for this assignment
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
  cell = {}
  if row.span.text=='Not assigned':
    pass
  else:
    cell['PostalCode'] = row.p.text[:3]
    cell['Borough'] = (row.span.text).split('(')[0]
    cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    table_contents.append(cell)

#print(table_contents)
#Parse table into a Data Frame

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({
    'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
    'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
    'EtobicokeNorthwest':'Etobicoke Northwest',
    'East YorkEast Toronto':'East York/East Toronto',
    'MississaugaCanada Post Gateway Processing Centre':'Mississauga'
})

Finaly we can merge both dataframes:

In [11]:
#Rename columns in the first dataframe
post_code_coord.columns = ['PostalCode', 'Latitude', 'Longitude']

#merge df using PostalCode reference
postal_code_info = pd.merge(df, post_code_coord, on='PostalCode')
postal_code_info.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
